## 第5章 顧客の退会を予測する10本ノック

### ノック41:データを読み込んで利用データを整形しよう

In [85]:
import pandas as pd

In [86]:
# 顧客データの読み込み
customer = pd.read_csv('sample_20201021/chapter_003/joined_customer.csv')

customer.head()

,Unnamed: 0,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routing_flg,calc_date,membership_period
0,0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [87]:
# 月ごとの利用データ
uselog_months = pd.read_csv('sample_20201021/chapter_004/uselog_months.csv')

uselog_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [88]:
year_months = list(uselog_months['年月'].unique())

uselog = pd.DataFrame()

for i in range(1, len(year_months)):

    tmp = uselog_months.loc[uselog_months['年月'] == year_months[i]].copy()
    tmp.rename(columns={'count': 'count_0'}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months['年月'] == year_months[i-1]].copy()

    del tmp_before['年月']
    
    tmp_before.rename(columns={'count': 'count_1'}, inplace=True)

    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    
    uselog = pd.concat([uselog, tmp], ignore_index=True)

uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### ノック42:退会前月の退会顧客データを作成しよう

In [89]:
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer['is_deleted'] == 1].copy()
exit_customer['exit_date'] = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])

for i in exit_customer.index:
    exit_customer.loc[i, 'exit_date'] = exit_customer.loc[i, 'end_date'] - relativedelta(months=1)

exit_customer['exit_date'] = pd.to_datetime(exit_customer['exit_date'])
exit_customer['年月'] = exit_customer['exit_date'].dt.strftime('%Y%m')

uselog['年月'] = uselog['年月'].astype(str)

exit_uselog = pd.merge(uselog, exit_customer, on=['customer_id', '年月'], how='left')

print(len(uselog))
print(len(exit_uselog))

exit_uselog.head()

33851
33851


,年月,customer_id,count_0,count_1,Unnamed: 0,name,class,gender,start_date,end_date,...,price,campaign_name,mean,median,max,min,routing_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [90]:
exit_uselog = exit_uselog.dropna(subset=['name'])

print(len(exit_uselog))
print(len(exit_uselog['customer_id'].unique()))

exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,Unnamed: 0,name,class,gender,start_date,end_date,...,price,campaign_name,mean,median,max,min,routing_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,2798.0,XXXXX,C01,M,2018-03-01,2018-06-30,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,2232.0,XX,C01,M,2017-11-01,2018-06-30,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,1752.0,XXXX,C01,M,2017-06-01,2018-06-30,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,2663.0,XXXXX,C01,M,2018-02-01,2018-06-30,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,2088.0,XX,C03,F,2017-09-01,2018-06-30,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


### ノック43:継続顧客のデータを作成しよう

In [91]:
continue_customer = customer.loc[customer['is_deleted'] == 0].copy()

continue_uselog = pd.merge(uselog, continue_customer, on=['customer_id'], how='left')

print(len(continue_uselog))

continue_uselog = continue_uselog.dropna(subset=['name'])

print(len(continue_uselog))

33851
27422


In [92]:
continue_uselog = continue_uselog.sample(frac=1, random_state=0).reset_index(drop=True)

print(len(continue_uselog))

continue_uselog = continue_uselog.drop_duplicates(subset='customer_id')

print(len(continue_uselog))

27422
2842


In [93]:
predict_data = pd.concat([continue_uselog, exit_uselog], ignore_index=True)

print(len(predict_data))

predict_data.head()

3946


,年月,customer_id,count_0,count_1,Unnamed: 0,name,class,gender,start_date,end_date,...,price,campaign_name,mean,median,max,min,routing_flg,calc_date,membership_period,exit_date
0,201806,AS015233,7,7.0,3162.0,XXXXX,C01,M,2018-05-13,NaT,...,10500.0,入会費半額,7.545455,7.0,11.0,4.0,1.0,2019-04-30,11.0,NaT
1,201812,AS046594,7,5.0,2791.0,XXX,C01,M,2018-03-01,NaT,...,10500.0,通常,7.166667,7.0,10.0,5.0,1.0,2019-04-30,13.0,NaT
2,201808,OA769163,6,4.0,1626.0,XXX,C01,M,2017-05-01,NaT,...,10500.0,通常,6.083333,6.0,8.0,2.0,1.0,2019-04-30,23.0,NaT
3,201805,AS981537,10,6.0,2623.0,XX,C03,M,2018-02-01,NaT,...,6000.0,通常,6.916667,7.0,10.0,5.0,1.0,2019-04-30,14.0,NaT
4,201805,GD065188,6,7.0,2109.0,XX,C01,M,2017-09-01,NaT,...,10500.0,通常,6.333333,6.5,9.0,3.0,1.0,2019-04-30,19.0,NaT


### ノック44:予測する月の在籍期間を作成しよう